In [ ]:
import pandas as pd

def convert_excel_to_csv(excel_file_path, csv_file_path):
    # Read the Excel file
    df = pd.read_excel(excel_file_path)

    # Save the dataframe to a CSV file
    df.to_csv(csv_file_path, index=False)

# Usage
excel_file_path = 'BADI_monthly_20Y.xlsx'
csv_file_path = 'BADI_monthly_20Y.csv'

convert_excel_to_csv(excel_file_path, csv_file_path)

In [ ]:
import pandas as pd

# File paths
file_path_data = './data/ffa/panamax/4tc_curmon_1mon.csv'
file_path_spot_prices = './data/spot/panamax/BAPI_historical.csv'

# Read the data CSV file
data_df = pd.read_csv(file_path_data, delimiter=';', parse_dates=['Row Labels'], dayfirst=True)
data_df.set_index('Row Labels', inplace=True)

# Read the spot prices CSV file
spot_prices_df = pd.read_csv(file_path_spot_prices, delimiter=';', parse_dates=['Date'], dayfirst=True)
spot_prices_df.set_index('Date', inplace=True)

# Ensure the index is a datetime
data_df.index = pd.to_datetime(data_df.index)
spot_prices_df.index = pd.to_datetime(spot_prices_df.index)

# Create a new index that contains all dates from both DataFrames
all_dates = data_df.index.union(spot_prices_df.index)

# Reindex the data DataFrame to include all dates, sorting the index afterwards
data_df = data_df.reindex(all_dates).sort_index()

# Replace missing values with NaN if any non-numeric data is present
data_df.replace('', pd.NA, inplace=True)

# Interpolate missing values linearly
data_df = data_df.interpolate(method='linear')

# Save the dataframe to a new CSV file
data_df.to_csv('./data/ffa/panamax/4tc_curmon_1mon_all_dates.csv', sep=';')

# Print the dataframe to verify
print(data_df)


In [2]:
import pandas as pd

# Load the dataset from the CSV file
df = pd.read_csv('./data/ffa/panamax/4tc_curmon_1mon_all_dates.csv', delimiter=';', parse_dates=['Date'], dayfirst=True)

# Ensure the 'Date' column is in datetime format (should be done by parse_dates, but just to be explicit)
df['Date'] = pd.to_datetime(df['Date'])

# Initialize '4TC_FORWARD' column with values from '4TC_PCURMON'
df['4TC_FORWARD'] = df['4TC_PCURMON']

# Find the last week of each month in the 'Date' column
last_week_of_month = df.groupby([df['Date'].dt.year, df['Date'].dt.month])['Date'].transform('max')

# If the date is in the last week of the month, use values from '4TC_P+1MON'
df.loc[df['Date'] >= (last_week_of_month - pd.Timedelta(days=7)), '4TC_FORWARD'] = df['4TC_P+1MON']

# Save the updated DataFrame to a new CSV file
df.to_csv('./data/ffa/panamax/4tc_forward.csv', sep=';', index=False)

# If you want to verify the output, you can print the first few rows of the DataFrame
print(df.head())


        Date  4TC_PCURMON  4TC_P+1MON  4TC_FORWARD
0 2009-09-10      20056.0     19683.0      20056.0
1 2009-09-11      19745.0     19160.0      19745.0
2 2009-09-14      19568.0     18960.0      19568.0
3 2009-09-15      19335.0     18458.0      19335.0
4 2009-09-16      19568.0     18661.0      19568.0
